# Case 3 | Ruin Theory

Actuarial Science Undergratuate Program | Business Faculty | Risk Theory Class | José Enrique Pérez Salvador

A policy covers physical damage incurred by the trucks in a company’s fleet. The number of losses in a year has a Poisson distribution with $ \lambda = 5 $. The amount of a single loss has a gamma distribution with $ \alpha = 0.5 $ and $ \theta=2,500 $. The insurance contract pays a maximum annual benefit of 20,000.

## ❓1) Determine the probability that the maximum benefit will be paid. Use a span of 100 and the method of rounding.

## ⏸️Method of Rounding

The easiest approach to construct a discrete severity distribution from a continuous one is to place the discrete probabilities on multiples of a convenient unit of measurement $ h $, the span.

**Method of rounding (mass dispersal)**: Let $ f_j $, denote the probability placed at $ jh $, $ j =0, 1,2, \dots $. Then set <br>
$ f_0 = P\left[ X < \frac{h}{2} \right] = F_X\left( \frac{h}{2} \right) $ <br>
$ f_j = P\left[jh - \frac{h}{2} \leq X < jh + \frac{h}{2} \right] = F_X\left( jh + \frac{h}{2} \right) - F_X\left( jh - \frac{h}{2} \right) ,j=1,2,\dots $

This method concentrates all the probability one-half span on either side of $ jh $ and places it at $ jh $. There is an exception for the probability assigned to zero. This, in effect, rounds all amounts to the nearest convenient monetary unit, $ h $, the span of the distribution. When the continuous severity distribution is unbounded, it is reasonable to halt the discretization process at some point once most all the probability has been accounted for. If the index for this last point is $ m $, then $ f_m = l-F_X\left[(m - 0.5)h \right] $. With this method the discrete
probabilities are never negative and sum to l, ensuring that the resulting distribution is
legitimate.

<div style="text-align: right">
Klugman, Stuart A. et. al.  (2012). Loss Models: from Data to Decisions, 4th ed.,USA: Wiley.
</div>

## ▶️Solution (1)

The parameterization of the gamma distribution in Loss Models is:<br>
- Probability density function: $ f(x)= \frac{(x/\theta)^\alpha \exp(-x/\theta)}{x\Gamma(\alpha)} $ <br>

The parameterization of the gamma distribution in SAS is:<br>
- Probability density function: $ f(x)= \frac{1}{\lambda^a \Gamma(a)} x^{a-1} \exp(-x/\lambda) $ <br>

🤷🏻‍♀️Do they use the same parameterization? Y/N

### Parameters

In [ ]:
* Loss distribution;
%let alpha=0.5;
%let theta=2500;
* Frequency distribution;
%let lambda=5;
* Discretization parameters of the loss distribution;
* Maximum value of the discrete loss random variable (m x h);
%let m=100;
* Span for the discretization;
%let h=100;
* In 100 units;
%let max_ben_h=200;
%put &=alpha &=theta &=lambda &=m &=h &=max_ben_h;

### Discretization of the gamma distribution

In [ ]:
data work.gamma_discrete;
    label
        j = "Index j"
        jh = " Amount associated to the index j"
        low = "If applies, lower limit of the interval"
        up = "If applies, upper limit of the interval"
        fj = "P[Xd=jh]";
    do j=0 to &m;
        jh = j*&h.;
        low=max(j*&h.-&h./2,0);
        up=j*&h.+&h./2;
        if j=&m then 
            fj=1-cdf('GAMMA',low,&alpha,&theta);
        else
            fj=cdf('GAMMA',up,&alpha,&theta)-cdf('GAMMA',low,&alpha,&theta);
        output;
    end;
run;

### Table of the probability function

In [ ]:
title "Probability function of the discrete loss random variable";
proc print data = work.gamma_discrete label;
run;
title;

### Checklist of the probability function

In [ ]:
title "Checklist of the probability function";
proc means data=work.gamma_discrete n min max sum;
run;
title;

### Plot of the probability function

In [ ]:
title 'Discrete gamma severity distribution';
ods graphics / reset width=10in height=4.8in imagemap noborder;
proc sgplot data=work.gamma_discrete;
needle x=jh y=fj /lineattrs=(color=pink pattern=2 thickness=1) markers markerattrs=(color=purple SYMBOL=circlefilled);
xaxis grid;
yaxis grid;
run;
title;

## ⏸️Panjer's recursion

Based on the exercise 2 of homework 3, the Panjer's recursion is useful to get the distribution of the collective model $ S = \sum_{i=0}^N X_i $ when the severity distribution $ f_X(x) $ is defined on $ 0,1,2,\dots,m $ representing multiples of some convenient monetary unit and the frequency distribution is a member of the $ (a,b,1) $ class. The number $ m $ represents the largest possible payment and could be infinite.<br>
Further, in the case of the Poisson distribution, the Panjer's recursion formula reduces to:<br>
<center> $ f_S(x) = \frac{\lambda}{x} \sum_{y=1}^{\min(x,m)} y f_X(y) f_S(x-y),x=1,2,\dots $ </center><br>
with starting point: <br>
<center> $ f_S(0) = \exp(-\lambda [ 1-f_X(0) ]) $ </center>

<div style="text-align: right">
Klugman, Stuart A. et. al.  (2012). Loss Models: from Data to Decisions, 4th ed.,USA: Wiley.
</div>

## ▶️Solution (2)

### Implementation of the Panjer's recursion formula

In [ ]:
proc iml;
    use work.gamma_discrete;
    read all var _NUM_ into sev[colname=numVars];
    close work.gamma_discrete;

    start Panjer_Poisson(p, lambda);
    /* Input validation */
    if sum(p) > 1 | any(p < 0) then do;
      print "Error: p parameter is not a valid probability distribution";
      abort;
    end;
    if lambda * sum(p) > 727 then do;
      print "Error: Underflow condition detected";
      abort;
    end;

    /* Initialize variables */
    r = nrow(p); /* Length of p vector */
    f = exp(-lambda * sum(p)); /* Initial probability f(0) */
    cumul = f; /* Cumulative probability */
    s = 0; /* Iteration counter */

    /* Panjer recursion loop */
    do while (1);
      s = s + 1;
      m = min(s, r); /* Minimum of s and r */
      /* Compute the sum for the recursion */
      sum_term = 0;
      do k = 1 to m;
        sum_term = sum_term + (k * p[k] * f[s - k + 1]);
      end;
      last = (lambda / s) * sum_term;
      f = f // last; /* Append new probability to f */
      cumul = cumul + last; /* Update cumulative probability */

      /* Exit condition */
      if cumul > 0.99999999 then leave;
    end;

    return(f);
    finish;

    store module=_all_;

    * DON'T INCLUDE IN THE VECTOR P[X=0];
    myp=sev[,'fj'][2:nrow(sev)];
    
    fS = Panjer_Poisson(myp, &lambda.);
    collective=t(0:(nrow(fS)-1))||fS;
    
    * We send the results to a data set;
    create work.collective from collective;
    append from collective;
    close work.collective;  

quit;

### Formatting the results data set

In [ ]:
* We add an index;
proc datasets library=work nolist nodetails;
    modify collective;
    rename
    col1=s
    col2=fS;
    label
    s = "Loss amount (in 100 units)"
    fs = "P[S=s]";
quit;

In [ ]:
title "Probability function of the aggregate losses (in 100 units)";
proc print data=work.collective(obs=10) label;
run;
title;

### Checklist of the probability function

In [ ]:
title "Checklist of the probability function";
proc means data=work.collective n min max sum;
run;
title;

### Plot of the probability function

In [ ]:
title 'Distribution of the collective risk';
ods graphics / reset width=10in height=4.8in imagemap noborder;
proc sgplot data=work.collective;
needle x=s y=fS /lineattrs=(color=pink pattern=2 thickness=1) markers markerattrs=(color=purple SYMBOL=circlefilled);
xaxis grid;
yaxis grid;
run;
title;

### Answer

In [ ]:
title 'Probability that the maximum benefit will be paid';
proc sql;
    select sum(fs)
    from work.collective
    where s >= &max_ben_h
    ;
quit;
title;

## ❓2) Compute the expected value principle premium. Apply a 50% loading factor.

In [ ]:
%let load_fact=0.5;

In [ ]:
title "Expected value principle premium (in 100 units)";
proc sql;
    select sum(min(s,&max_ben_h)*fS)*(1+&load_fact) into: premium
    from work.collective
    ;
quit;
title;

In [ ]:
%put &=premium;

## ❓3) The initial surplus is 150 (in 100 units). Compute the ruin probabilities by time 1 year, 2 years and 3 years.

In [ ]:
%let u = 150;
%put &=u;

### Ruin probability by 1 year

In [ ]:
proc sql;
    create table work.ruin_1 as
    select 
    a.*
    , &u + &premium - s as u_1 label="Surplus at time 1"
    from work.collective a
    ;
quit;

In [ ]:
title "Probability function of the surplus at time 1";
proc print data=work.ruin_1(obs=10) label;
run;
title;

In [ ]:
title "Checklist of the probability function";
proc means data=work.ruin_1 n min max sum;
run;
title;

In [ ]:
title 'Distribution of the surplus at time 1';
ods graphics / reset width=10in height=4.8in imagemap noborder;
proc sgplot data=work.ruin_1;
needle x=u_1 y=fS /lineattrs=(color=yellow pattern=2 thickness=1) markers markerattrs=(size=1pt color=green SYMBOL=circlefilled);
xaxis grid;
yaxis grid;
run;
title;

In [ ]:
title "Ruin probability by 1 year";
proc sql;
    select sum(fS) into: pr1
    from work.ruin_1
    where u_1 <= 0
    ;
quit;
title;

In [ ]:
%put &=pr1;

### Ruin probability by 2 years

In [ ]:
proc sql;
    create table work.ruin_2 as
    select 
    a.*
    , b.s as s2
    , b.fS as fS2
    , a.fS*b.fS as fu_2
    , a.u_1 + &premium - b.s as u_2 label="Surplus at time 2"
    from work.ruin_1 a, work.collective b
    ;
quit;

In [ ]:
title "Probability function of the surplus at time 2";
proc print data=work.ruin_2(obs=10) label;
run;
title;

In [ ]:
title "Checklist of the probability function";
proc means data=work.ruin_2 n min max sum;
run;
title;

In [ ]:
proc sql;
    create table work.ruin_2_plot as
    select
    round(u_2,0.01) as u_2 label="Surplus at time 2"
    , sum(fu_2) as fu_2
    from work.ruin_2
    group by round(u_2,0.01)
    ;
quit;

In [ ]:
title 'Distribution of the surplus at time 2';
ods graphics / reset width=10in height=4.8in imagemap noborder;
proc sgplot data=work.ruin_2_plot;
needle x=u_2 y=fu_2 /lineattrs=(color=yellow pattern=2 thickness=1) markers markerattrs=(size=1pt color=green SYMBOL=circlefilled);
xaxis grid;
yaxis grid;
run;
title;

In [ ]:
title "Ruin probability by 2 years";
proc sql;
    select sum(fu_2) + &pr1 into: pr2
    from work.ruin_2
    where u_2 <= 0 and u_1 > 0
    ;
quit;
title;

### Ruin probability by 3 years

In [ ]:
proc sql;
    create table work.ruin_3 as
    select 
    a.*
    , b.s as s3
    , b.fS as fS3
    , a.fu_2*b.fS as fu_3
    , a.u_2 + &premium - b.s as u_3 label="Surplus at time 3"
    from work.ruin_2 a, work.collective b
    ;
quit;

In [ ]:
title "Probability function of the surplus at time 3";
proc print data=work.ruin_3(obs=10) label;
run;
title;

In [ ]:
title "Checklist of the probability function";
proc means data=work.ruin_3 n min max sum;
run;
title;

In [ ]:
proc sql;
    create table work.ruin_3_plot as
    select
    round(u_3,0.01) as u_3 label="Surplus at time 3"
    , sum(fu_3) as fu_3
    from work.ruin_3
    group by round(u_3,0.01)
    ;
quit;

In [ ]:
title 'Distribution of the surplus at time 3';
ods graphics / reset width=10in height=4.8in imagemap noborder;
proc sgplot data=work.ruin_3_plot;
needle x=u_3 y=fu_3 /lineattrs=(color=yellow pattern=2 thickness=1) markers markerattrs=(size=1pt color=green SYMBOL=circlefilled);
xaxis grid;
yaxis grid;
run;
title;

In [ ]:
title "Ruin probability by 3 years";
proc sql;
    select sum(fu_3) + &pr2 into: pr3
    from work.ruin_3
    where u_3 <= 0 and u_2 > 0 and u_1 > 0 
    ;
quit;
title;

In [ ]:
%put &=pr1. &=pr2. &=pr3. 